## Получение institution_id "Bank of America"

In [1017]:
import json
import requests
from plaid import Client
pub_key="b27964897dfd9b8f31451dd23752d9"
# Поиск ИД Фин института через
# https://documenter.getpostman.com/view/4675947/RWMLHkHR?version=latest#399f481a-2fcc-446b-a269-1937317fdef2
url_inst_id='https://sandbox.plaid.com/institutions/search'
url_inst_body={"query": "Bank of America",    "products": ["transactions"],    "public_key": pub_key}
header = {"Content-Type": "application/json","User-Agent": "Plaid Postman"}
inst_request=requests.post(url_inst_id,data=json.dumps(url_inst_body),headers=header)
inst_id = inst_request.json()['institutions'][0]['institution_id']

## Генерация Public и Access токена

In [1018]:
# https://documenter.getpostman.com/view/4675947/RWMLHkHR?version=latest#92942453-9874-4536-92ae-18fd2edcc5d6
url_pub_token='https://sandbox.plaid.com/sandbox/public_token/create'
pub_key_body = ({
  "public_key": pub_key,
  "institution_id": inst_id,
  "initial_products": ["assets"],
  "options": {
    "webhook": "https://www.genericwebhookurl.com/webhook"
  }
})
pub_token_request=requests.post(url_pub_token,data=json.dumps(pub_key_body),headers=header)
pub_token = pub_token_request.json()['public_token']

# https://github.com/plaid/plaid-python#examples 
client = Client(client_id='5ecb70c91aa68500131ccd63', secret='8d81c7a27f64aa26f4733c2c87fc87', public_key='b27964897dfd9b8f31451dd23752d9', environment='sandbox')
response = client.Item.public_token.exchange(pub_token)
access_token = response['access_token']
# Здесь идёт получение public_token и access_token, чтобы по ним сгенерировать списки транзакций и Asset report

## Получение списка транзакций за период

In [1020]:
import pandas as pd
import time
start_date='2020-03-01'
end_date='2020-05-27'
# time.sleep(30)
transactions_data = client.Transactions.get(access_token, start_date=start_date, end_date=end_date)
categories = client.Categories.get()
df_transactions=pd.DataFrame.from_dict(transactions_data['transactions'])
df_accounts=pd.DataFrame.from_dict(transactions_data['accounts'])
df_categories=pd.DataFrame.from_dict(categories['categories'])
# Из всего этого будет использоваться только df_accounts

## Данные по транзакциям решил взять из Asset report

In [1021]:
asset_rep_cred=client.AssetReport.create([access_token],57,options=None)
# time.sleep(30)
asset_rep_token=asset_rep_cred['asset_report_token']
assrep=client.AssetReport.get(asset_rep_token)
# отчёт за последние 57 дней (с начала апреля)

In [1022]:
a=[]
for i in assrep['report']['items'][0]['accounts']:
    for j in i['transactions']:
        a.append(j)
# Получение из Asset_report только списка транзакций

In [1023]:
asset_dfraw=pd.DataFrame.from_dict(a)

In [1024]:
asset_dfmerged=pd.merge(asset_dfraw,df_accounts, how='left', on='account_id')
asset_dfclean=asset_dfmerged.drop(['account_id','iso_currency_code','pending','transaction_id','unofficial_currency_code','balances','mask','name','official_name','subtype'], axis=1)

In [1025]:
asset_dfclean['Income/Expense']=asset_dfclean['amount'].apply(lambda x: 'Income' if x>=0 else 'Expense')
asset_dfclean['month'] =  pd.to_datetime(asset_dfclean['date'], format='%Y-%m-%d').dt.strftime('%b %Y')
asset_dfclean=asset_dfclean.drop(['date'],axis=1)
# Подготовка датафрейма

In [1026]:
out_df = asset_dfclean.groupby(['Income/Expense']).apply(lambda sub: sub.pivot_table(
    index=['Income/Expense','original_description','type'],
    values='amount', 
    columns='month',
    aggfunc=sum,
    margins=True,
    margins_name='Total',
))

out_df.index = out_df.index.droplevel(0)
out_df=out_df.reset_index()
# Пивот и сброс мультииндекса

## Экспорт в GooglespreadSheets

In [1027]:
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'myproject001xo-2fd41c5157cf.json', scope)
gc = gspread.authorize(credentials)
# Настройка доступа, через json файл от Google

In [1030]:
spreadsheet_key = '1mZ0exUHQGoA7Vtc6-vWhJwpE8NZL_KkCbdvsQ-Y63Gs'
wks_name = 'Raw'
d2g.upload(asset_dfraw, spreadsheet_key, wks_name, credentials=credentials, row_names=True)
# Отправка сырого DF 

<Worksheet 'Raw' id:0>

In [1031]:
spreadsheet_key = '1mZ0exUHQGoA7Vtc6-vWhJwpE8NZL_KkCbdvsQ-Y63Gs'
wks_name = 'Formated'
d2g.upload(out_df, spreadsheet_key, wks_name, credentials=credentials, row_names=True)
# Отправка готового DF

<Worksheet 'Formated' id:1011452588>

P.s. не сделал Subtotal для каждой группы, т.к. не хватило времени с ними разобраться
Ссылка на Google Spreadsheet:
https://docs.google.com/spreadsheets/d/1mZ0exUHQGoA7Vtc6-vWhJwpE8NZL_KkCbdvsQ-Y63Gs/edit#gid=0